## Pre-trained word embeddings

In [12]:
import gensim.downloader as api
from scipy.spatial.distance import cosine
import numpy as np

# Pre-trained Word2Vec model from google
# This model is huge and takes a while to download 10+ mins for me (sorry)
model = api.load("word2vec-google-news-300")


In [29]:
# Calculate cosine similarity between two words using pre-trained Word2Vec embeddings
def cosine_similarity(word1, word2):
    
    # Check if both words are present in the vocabulary
    if word1 in model and word2 in model:
    
        return 1 - cosine(model[word1], model[word2])

    else:
        print("One or both of those words isn't in the data set")
        return None

In [14]:
# Calculate dissimilarity score between two words based on
def dissimilarity_score(word1, word2):
    
    # Check if both words are present in the vocabulary
    if word1 in model and word2 in model:

        print("not implemented")
    
        return 0

    else:
        print("One or both of those words isn't in the data set")
        return None

In [17]:
# Get user input
word1 = input("Enter the first word: ")
word2 = input("Enter the second word: ")

cosine_sim = cosine_similarity(word1, word2)
if cosine_sim is not None:
    print(f"Cosine Similarity between '{word1}' and '{word2}': {cosine_sim}")

dissimilarity = dissimilarity_score(word1, word2)
if dissimilarity is not None:
    print(f"Dissimilarity Score between '{word1}' and '{word2}': {dissimilarity}")


Cosine Similarity between 'the' and 'the': 1.0
Dissimilarity Score between 'the' and 'the': 0.0


The dissimilarity score should be defined by you. You either can
have your own idea of a dissimilarity score or refer to literature (cite the paper you used). In
either case clearly describe how this score helps determine the dissimilarity between 2 words

Note: Dissimilarity measure has been an important metric for recommender systems trying to
introduce ‘Novelty and Diversity’ in assortments (as opposed to only accuracy). You might find
different metrics of dissimilarity in recommender system’s literature.